## Import Libraries

In [17]:
import requests
import pandas as pd
import numpy as np
import time



## Load Wallet Addresses and API Configuration

In [30]:
API_KEY = "YOUR_API_KEY_HERE"  # Replace with your actual key when running
CHAIN_ID = "1"  # Ethereum Mainnet



In [19]:
# Load wallets from CSV
wallet_df = pd.read_csv("Wallet id - Sheet1.csv")
wallets = wallet_df["wallet_id"].tolist()


## Define Compound V2 Contracts for Filtering

In [20]:
compound_contracts = [
    "0x3d9819210a31b4961b30ef54be2aed79b9c9cd3b",  # Comptroller
    "0x5d3a536e4d6dbd6114cc1ead35777bab948e3643",  # cDAI
    "0x39aa39c021dfbae8fac545936693ac917d5e7563",  # cUSDC
    "0x6c8c6b02e7b2be14d4fa6022dfd6d9d9e6e0b35c",  # cBAT
    "0x158079Ee67Fce2f58472A96584A73C7Ab9AC95c1",  # cREP
    "0x6d7f0754ffeb405d23c51ce938289d4835be3b14",  # cWBTC
    "0x35a18000230da775cac24873d00ff85bccded550",  # cUNI
    "0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5",  # cETH
    "0x95b4ef2869ebd94beb4eee400a99824bf5dc325b",  # cLINK
    "0xface851a4921ce59e912d19329929ce6da6eb0c7",  # cCOMP
    "0x41b5844f4680dbc3820f7ba2e826cb990b479e75",  # cSUSHI
    "0x4a92e71227d294f041bd82dd8f78591b75140d63"   # cZRX
]


## Fetch Compound Transaction History for a Wallet

In [21]:
def get_compound_activity(wallet):
    """
    Fetch Compound Protocol activity for a given wallet using Covalent.
    """
    contracts = ",".join(compound_contracts)
    url = f"https://api.covalenthq.com/v1/{CHAIN_ID}/address/{wallet}/transactions_v2/"

    params = {
        "contract-address": contracts,
        "key": API_KEY
    }

    try:
        resp = requests.get(url, params=params)
        if resp.status_code != 200:
            print(f"❌ Error for {wallet}: {resp.status_code}")
            return None
        data = resp.json()["data"]["items"]
        return data
    except Exception as e:
        print(f"⚠️ Exception for {wallet}: {e}")
        return None


## Extract Risk-Related Features from Transaction Data

In [22]:
def extract_features(wallet_data):
    """
    Extract relevant financial risk features from wallet's transaction history.
    Safely handles missing or malformed log_events.
    """
    if not wallet_data:
        return {
            "tx_count": 0,
            "borrow_tx": 0,
            "repay_tx": 0,
            "liquidations": 0,
            "unique_tokens": 0,
            "net_activity": 0
        }

    tx_count = len(wallet_data)
    borrow_tx = 0
    repay_tx = 0
    liquidations = 0
    unique_tokens = set()

    for tx in wallet_data:
        # Skip if no logs
        log_events = tx.get("log_events", [])
        for log in log_events:
            decoded = log.get("decoded")
            if not decoded:
                continue
            method = decoded.get("name", "").lower()
            if "borrow" in method:
                borrow_tx += 1
            elif "repay" in method:
                repay_tx += 1
            elif "liquidate" in method:
                liquidations += 1

        # Track tokens interacted with (if any)
        for transfer in tx.get("transfers", []):
            contract_address = transfer.get("contract_address")
            if contract_address:
                unique_tokens.add(contract_address.lower())

    net_activity = borrow_tx - repay_tx - liquidations

    return {
        "tx_count": tx_count,
        "borrow_tx": borrow_tx,
        "repay_tx": repay_tx,
        "liquidations": liquidations,
        "unique_tokens": len(unique_tokens),
        "net_activity": net_activity
    }


## Loop Through Wallets and Collect Features

In [23]:
# Loop through wallets
features_list = []
for wallet in wallets:
    print(f"Processing {wallet}")
    data = get_compound_activity(wallet)
    features = extract_features(data)
    features["wallet_id"] = wallet
    features_list.append(features)
    time.sleep(0.3)  # Respect API rate limits

# Create DataFrame
df = pd.DataFrame(features_list)

Processing 0x0039f22efb07a647557c7c5d17854cfd6d489ef3
Processing 0x06b51c6882b27cb05e712185531c1f74996dd988
Processing 0x0795732aacc448030ef374374eaae57d2965c16c
Processing 0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
Processing 0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
Processing 0x104ae61d8d487ad689969a17807ddc338b445416
Processing 0x111c7208a7e2af345d36b6d4aace8740d61a3078
Processing 0x124853fecb522c57d9bd5c21231058696ca6d596
Processing 0x13b1c8b0e696aff8b4fee742119b549b605f3cbc
Processing 0x1656f1886c5ab634ac19568cd571bc72f385fdf7
Processing 0x1724e16cb8d0e2aa4d08035bc6b5c56b680a3b22
Processing 0x19df3e87f73c4aaf4809295561465b993e102668
Processing 0x1ab2ccad4fc97c9968ea87d4435326715be32872
Processing 0x1c1b30ca93ef57452d53885d97a74f61daf2bf4f
Processing 0x1e43dacdcf863676a6bec8f7d6896d6252fac669
Processing 0x22d7510588d90ed5a87e0f838391aaafa707c34b
Processing 0x24b3460622d835c56d9a4fe352966b9bdc6c20af
Processing 0x26750f1f4277221bdb5f6991473c6ece8c821f9d
Processing 0x27f72a000d8e9f3

In [24]:
print("📋 Column Names:")
print(df.columns.tolist())


📋 Column Names:
['tx_count', 'borrow_tx', 'repay_tx', 'liquidations', 'unique_tokens', 'net_activity', 'wallet_id']


## Normalize Features and Compute Risk Scores

In [25]:
# Normalize and score
def min_max_normalize(series):
    if series.max() == series.min():
        return pd.Series([0.5] * len(series))
    return (series - series.min()) / (series.max() - series.min())

In [26]:
df["score"] = (
    min_max_normalize(df["tx_count"]) * 0.2 +
    min_max_normalize(df["borrow_tx"]) * 0.25 +
    min_max_normalize(df["repay_tx"]) * 0.15 +
    (1 - min_max_normalize(df["liquidations"])) * 0.25 +
    min_max_normalize(df["unique_tokens"]) * 0.1 +
    min_max_normalize(df["net_activity"]) * 0.05
) * 1000


In [27]:
df["score"] = df["score"].round().astype(int)

## Save Results to CSV

In [29]:
# Output
final_df = df[["wallet_id", "score"]]
final_df.to_csv("wallet_risk_scores.csv", index=False)
print("Scoring complete. Output saved to 'wallet_risk_scores.csv'")

Scoring complete. Output saved to 'wallet_risk_scores.csv'
